In [ ]:
root_path =  "/content/drive/MyDrive/CDS_project/Team No Name/"  

In [ ]:
import os
import pandas as pd
import numpy as np
csv_path = os.path.join(root_path,'UrbanSound8K/metadata/UrbanSound8K.csv')
spectrograms_path = os.path.join(root_path,'numpySpectrograms/')
test_size = 0.2

# Using orignal spectrogram

In [ ]:
from tqdm import tqdm

class SpecLoader():
  def __init__(self, x_set, spec_dir):
    self.x = x_set
    self.spec_dir = spec_dir

  def __len__(self):
    return len(self.x)

  def getitem(self):
    specs = []
    for fileName in tqdm(self.x):
        spec = np.load(self.spec_dir + fileName + ".npy")
        specs.append(spec.transpose())
    return np.array(specs)

  def getitemlist(self):
    specs = []
    for fileName in tqdm(self.x):
        spec = np.load(self.spec_dir + fileName + ".npy")
        specs.append(spec.transpose())
    return specs

In [ ]:
from sklearn.model_selection import train_test_split

known_test_size = 0.2

data_df = pd.read_csv(csv_path)
data_df_known = data_df.loc[data_df["classID"] < 5]
data_df_unknown = data_df.loc[data_df["classID"] >= 5]

Xtrain_known, Xtest_known, Ytrain_known, Ytest_known = train_test_split(data_df_known['slice_file_name'].tolist(), data_df_known['classID'].tolist(), test_size=known_test_size, random_state = 42)


# separate for diff models
xtrain0, xtrain1, xtrain2, xtrain3, xtrain4 = [], [], [], [], []

for i in range(len(Xtrain_known)):
  if Ytrain_known[i] == 0:
    xtrain0.append(Xtrain_known[i])
  elif Ytrain_known[i] == 1:
    xtrain1.append(Xtrain_known[i])
  elif Ytrain_known[i] == 2:
    xtrain2.append(Xtrain_known[i])
  elif Ytrain_known[i] == 3:
    xtrain3.append(Xtrain_known[i])
  elif Ytrain_known[i] == 4:
    xtrain4.append(Xtrain_known[i]) 
  else:
    print("cannot separate")

len0 = len(xtrain0)
len1 = len(xtrain1)
len2 = len(xtrain2)
len3 = len(xtrain3)
len4 = len(xtrain4)


xList = xtrain0 + xtrain1 + xtrain2 + xtrain3 + xtrain4
loader = SpecLoader(xList, spectrograms_path)
xtrainsSpec = loader.getitemlist()

x0 = xtrainsSpec[:len0]
x1 = xtrainsSpec[len0:len0+len1]
x2 = xtrainsSpec[len0+len1:len0+len1+len2]
x3 = xtrainsSpec[len0+len1+len2:len0+len1+len2+len3]
x4 = xtrainsSpec[len0+len1+len2+len3:]

X0trainval = x0 + x1 + x2 + x3 + x4
Y0trainval = [1 for i in range(len0)]+[-1 for i in range(len1+len2+len3+len4)]
X1trainval = x1 + x0 + x2 + x3 + x4
Y1trainval = [1 for i in range(len1)]+[-1 for i in range(len0+len2+len3+len4)]
X2trainval = x2 + x0 + x1 + x3 + x4
Y2trainval = [1 for i in range(len2)]+[-1 for i in range(len0+len1+len3+len4)]
X3trainval = x3 + x0 + x2 + x1 + x4
Y3trainval = [1 for i in range(len3)]+[-1 for i in range(len0+len2+len1+len4)]
X4trainval = x4 + x0 + x2 + x3 + x1
Y4trainval = [1 for i in range(len4)]+[-1 for i in range(len0+len2+len3+len1)]


100%|██████████| 3543/3543 [49:35<00:00,  1.19it/s]


In [ ]:
# converting to numpy
X0train = np.array(X0trainval)
nsamples, x, y, z= X0train.shape
X0train = X0train.reshape((nsamples, x*y*z))
Y0train = np.array(Y0trainval)

X1train = np.array(X1trainval)
nsamples, x, y, z= X1train.shape
X1train = X1train.reshape((nsamples, x*y*z))
Y1train = np.array(Y1trainval)

X2train = np.array(X2trainval)
nsamples, x, y, z= X2train.shape
X2train = X2train.reshape((nsamples, x*y*z))
Y2train = np.array(Y2trainval)

X3train = np.array(X3trainval)
nsamples, x, y, z= X3train.shape
X3train = X3train.reshape((nsamples, x*y*z))
Y3train = np.array(Y3trainval)

X4train = np.array(X4trainval)
nsamples, x, y, z= X4train.shape
X4train = X4train.reshape((nsamples, x*y*z))
Y4train = np.array(Y4trainval)

In [ ]:
print(X0train.shape)
print(Y0train.shape)

(3543, 96000)
(3543,)


In [ ]:
# save pickle file
import pickle
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
# from sklearn.externals.joblib import parallel_backend

path = "/content/drive/MyDrive/CDS_project/Team No Name/OneVsAll/original_spec/"

kernel = "sigmoid"
param_grid  = {'C': [1, 10, 100, 1000], 
              'kernel': [kernel]}  

 
grid_0 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
grid_0.fit(X0train, Y0train)
grid_0.dump(grid_0, open(path+kernel+"0.pkl", "wb"))


grid_1 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
grid_1.fit(X1train, Y1train)
pickle.dump(grid_1, open(path+kernel+"1.pkl", "wb"))

  
grid_2 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
grid_2.fit(X2train, Y2train)
pickle.dump(grid_2, open(path+kernel+"2.pkl", "wb"))


grid_3 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
grid_3.fit(X3train, Y3train)
pickle.dump(grid_3, open(path+kernel+"3.pkl", "wb"))


grid_4 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
grid_4.fit(X4train, Y4train)
pickle.dump(grid_4, open(path+kernel+"4.pkl", "wb"))


print("########## Done #############")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] C=1, kernel=sigmoid .............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................. C=1, kernel=sigmoid, score=0.709, total=34.8min
[CV] C=1, kernel=sigmoid .............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 34.8min remaining:    0.0s


[CV] ................. C=1, kernel=sigmoid, score=0.619, total=34.6min
[CV] C=1, kernel=sigmoid .............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 69.4min remaining:    0.0s


[CV] ................. C=1, kernel=sigmoid, score=0.757, total=35.3min
[CV] C=1, kernel=sigmoid .............................................
[CV] ................. C=1, kernel=sigmoid, score=0.802, total=34.8min
[CV] C=1, kernel=sigmoid .............................................
[CV] ................. C=1, kernel=sigmoid, score=0.715, total=35.4min
[CV] C=10, kernel=sigmoid ............................................
[CV] ................ C=10, kernel=sigmoid, score=0.701, total=33.3min
[CV] C=10, kernel=sigmoid ............................................
[CV] ................ C=10, kernel=sigmoid, score=0.594, total=31.7min
[CV] C=10, kernel=sigmoid ............................................
[CV] ................ C=10, kernel=sigmoid, score=0.760, total=33.9min
[CV] C=10, kernel=sigmoid ............................................
[CV] ................ C=10, kernel=sigmoid, score=0.795, total=33.4min
[CV] C=10, kernel=sigmoid ............................................
[CV] .

# Using features extracted from base classification model

In [ ]:
import numpy as np
import pandas as pd

import json
import keras
import tensorflow
from keras import layers
from keras.models import Sequential
from keras.applications import DenseNet201
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

from keras.models import load_model

In [ ]:
model_save_path = os.path.join(root_path,"basemodel-known")
model = load_model(model_save_path)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, None, None, 1920)  18321984  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1920)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 9605      
Total params: 18,331,589
Trainable params: 18,102,533
Non-trainable params: 229,056
_________________________________________________________________


In [ ]:
batch_size = 16

In [ ]:
class SpecLoaderWnn(keras.utils.Sequence):
  def __init__(self, x_set, batch_size, spec_dir):
    self.x = x_set
    self.batch_size = batch_size
    self.spec_dir = spec_dir

  def __len__(self):
    return int(np.ceil(len(self.x) / self.batch_size))

  def getitem(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
    # batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

    batchSpecs = []
    for fileName in batch_x:
        spec = np.load(self.spec_dir + fileName + ".npy")
        batchSpecs.append(spec.transpose())
    return np.array(batchSpecs)


In [ ]:
#prepare data
from sklearn.model_selection import train_test_split

data_df = pd.read_csv(csv_path)
data_df_known = data_df.loc[data_df["classID"] < 5]
data_df_unknown = data_df.loc[data_df["classID"] >= 5]

Xtrain_known, Xtest_known, Ytrain_known, Ytest_known = train_test_split(data_df_known['slice_file_name'].tolist(), data_df_known['classID'].tolist(), test_size=0.2, random_state = 42)


# separate for diff models
xtrain0, xtrain1, xtrain2, xtrain3, xtrain4 = [], [], [], [], []

for i in range(len(Xtrain_known)):
  if Ytrain_known[i] == 0:
    xtrain0.append(Xtrain_known[i])
  elif Ytrain_known[i] == 1:
    xtrain1.append(Xtrain_known[i])
  elif Ytrain_known[i] == 2:
    xtrain2.append(Xtrain_known[i])
  elif Ytrain_known[i] == 3:
    xtrain3.append(Xtrain_known[i])
  elif Ytrain_known[i] == 4:
    xtrain4.append(Xtrain_known[i]) 
  else:
    print("cannot separate")

len0 = len(xtrain0)
len1 = len(xtrain1)
len2 = len(xtrain2)
len3 = len(xtrain3)
len4 = len(xtrain4)



train_loader = SpecLoaderWnn(Xtrainval, Ytrainval, batch_size, spectrograms_path)

In [ ]:
from keras import Model
from tqdm import tqdm

## extract feature from base model
xList = xtrain0 + xtrain1 + xtrain2 + xtrain3 + xtrain4
train_loader = SpecLoaderWnn(xList, batch_size, spectrograms_path)

finaltrainingset = None
model_output = model.get_layer("global_average_pooling2d_1").output
m = Model(inputs=model.input, outputs=model_output)
for ind in tqdm(range(len(train_loader))):
  finaloutput = m.predict(train_loader.getitem(ind))
  if ind==0:
    print(finaloutput.shape)
  if type(finaltrainingset) == type(None):
    finaltrainingset = finaloutput
    print("if type")
  else:
    finaltrainingset=np.concatenate((finaltrainingset, finaloutput),axis=0)



  0%|          | 1/222 [00:23<1:27:14, 23.69s/it]

(16, 1920)
if type


100%|██████████| 222/222 [1:10:54<00:00, 19.16s/it]


In [ ]:
print(len(xList))
print(len(finaltrainingset))


x0 = finaltrainingset[:len0]
x1 = finaltrainingset[len0:len0+len1]
x2 = finaltrainingset[len0+len1:len0+len1+len2]
x3 = finaltrainingset[len0+len1+len2:len0+len1+len2+len3]
x4 = finaltrainingset[len0+len1+len2+len3:]


X0trainval = np.concatenate((x0, x1, x2, x3, x4))
Y0trainval = [1 for i in range(len0)]+[-1 for i in range(len1+len2+len3+len4)]
X1trainval = np.concatenate((x1, x0, x2, x3, x4))
Y1trainval = [1 for i in range(len1)]+[-1 for i in range(len0+len2+len3+len4)]
X2trainval = np.concatenate((x2, x0, x1, x3, x4))
Y2trainval = [1 for i in range(len2)]+[-1 for i in range(len0+len1+len3+len4)]
X3trainval = np.concatenate((x3, x0, x2, x1, x4))
Y3trainval = [1 for i in range(len3)]+[-1 for i in range(len0+len2+len1+len4)]
X4trainval = np.concatenate((x4, x0, x2, x3, x1))
Y4trainval = [1 for i in range(len4)]+[-1 for i in range(len0+len2+len3+len1)]

3543
3543


In [ ]:
# fit into grid fit 
X0train = X0trainval
Y0train = np.array(Y0trainval)

X1train = X1trainval
Y1train = np.array(Y1trainval)

X2train = X2trainval
Y2train = np.array(Y2trainval)

X3train = X3trainval
Y3train = np.array(Y3trainval)

X4train = X4trainval
Y4train = np.array(Y4trainval)

grid.**fit**(X, y=None, *, groups=None, **fit_params) <br>
X: array-like of shape (n_samples, n_features)<br>
y: array-like of shape (n_samples, n_output) or (n_samples,), default=None




In [ ]:
# save pickle file
import pickle
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

path = "/content/drive/MyDrive/Colab Notebooks/base_model/"


kernel = "rbf" 
param_grid  = {'C': [0.1, 1, 10, 100, 1000], 
              'kernel': [kernel]}  


grid0 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3, n_jobs=-1) 
grid0.fit(X0train, Y0train)
pickle.dump(grid0, open(path+kernel+"0.pkl", "wb"))


grid1 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3, n_jobs=-1) 
grid1.fit(X1train, Y1train)
pickle.dump(grid1, open(path+kernel+"1.pkl", "wb"))


grid2 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3, n_jobs=-1) 
grid2.fit(X2train, Y2train)
pickle.dump(grid2, open(path+kernel+"2.pkl", "wb"))


grid3 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3, n_jobs=-1) 
grid3.fit(X3train, Y3train)
pickle.dump(grid3, open(path+kernel+"3.pkl", "wb"))


grid4 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3, n_jobs=-1) 
grid4.fit(X4train, Y4train)
pickle.dump(grid4, open(path+kernel+"4.pkl", "wb"))


print("########## Done #############")

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  1.6min finished


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  1.2min finished


Fitting 5 folds for each of 5 candidates, totalling 25 fits


...

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.5min finished


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  1.9min finished


########## Done #############


## Test and Evalute features from base model

In [ ]:
## testing
### split to get unknwon
X_trash, Xtest_unknown, y_trash, Ytest_unknown = train_test_split(data_df_unknown['slice_file_name'].tolist(), data_df_unknown['classID'].tolist(), test_size=0.2, random_state = 42)

testX = Xtest_unknown + Xtest_known

testY = [-1 for i in Xtest_unknown]+[1 for i in Xtest_known]
testY = np.array(testY)
testY_class = [-1 for i in Ytest_unknown] + Ytest_known


####### get test features from base model
from keras import Model
from tqdm import tqdm

test_loader = SpecLoaderWnn(testX, batch_size, spectrograms_path)

finaltestset = None
model_output = model.get_layer("global_average_pooling2d_1").output
m = Model(inputs=model.input, outputs=model_output)
for ind in tqdm(range(len(test_loader))):
  finaloutput = m.predict(test_loader.getitem(ind))
  if ind==0:
    print(finaloutput.shape)
  if type(finaltestset) == type(None):
    finaltestset = finaloutput
    print("if type")
  else:
    finaltestset=np.concatenate((finaltestset, finaloutput),axis=0)


  1%|▏         | 1/67 [00:20<22:51, 20.78s/it]

(16, 1920)
if type


100%|██████████| 67/67 [20:26<00:00, 18.31s/it]


In [ ]:
# load pickle
poly0 = pd.read_pickle(path+"linear0.pkl")
lin1 = pd.read_pickle(path+"linear1.pkl")
lin2 = pd.read_pickle(path+"linear2.pkl")
lin3 = pd.read_pickle(path+"linear3.pkl")
lin4 = pd.read_pickle(path+"linear4.pkl")

# -1 or 1
lpred0 = lin0.predict(finaltestset)
lpred1 = lin1.predict(finaltestset)
lpred2 = lin2.predict(finaltestset)
lpred3 = lin3.predict(finaltestset)
lpred4 = lin4.predict(finaltestset)

lprob_y0 = lin0.predict_proba(finaltestset)
lprob_y1 = lin1.predict_proba(finaltestset)
lprob_y2 = lin2.predict_proba(finaltestset)
lprob_y3 = lin3.predict_proba(finaltestset)
lprob_y4 = lin4.predict_proba(finaltestset)


# ---------  poly -----------------
poly0 =pd.read_pickle(path+"poly0.pkl")
poly1 = pd.read_pickle(path+"poly1.pkl")
poly2 = pd.read_pickle(path+"poly2.pkl")
poly3 = pd.read_pickle(path+"poly3.pkl")
poly4 = pd.read_pickle(path+"poly4.pkl")

# -1 or 1
ppred0 = poly0.predict(finaltestset)
ppred1 = poly1.predict(finaltestset)
ppred2 = poly2.predict(finaltestset)
ppred3 = poly3.predict(finaltestset)
ppred4 = poly4.predict(finaltestset)

pprob_y0 = poly0.predict_proba(finaltestset)
pprob_y1 = poly1.predict_proba(finaltestset)
pprob_y2 = poly2.predict_proba(finaltestset)
pprob_y3 = poly3.predict_proba(finaltestset)
pprob_y4 = poly4.predict_proba(finaltestset)

# ---------  rbf -----------------
rbf0 =pd.read_pickle(path+"rbf0.pkl")
rbf1 = pd.read_pickle(path+"rbf1.pkl")
rbf2 = pd.read_pickle(path+"rbf2.pkl")
rbf3 = pd.read_pickle(path+"rbf3.pkl")
rbf4 = pd.read_pickle(path+"poly4.pkl")

# -1 or 1
rpred0 = rbf0.predict(finaltestset)
rpred1 = rbf1.predict(finaltestset)
rpred2 = rbf2.predict(finaltestset)
rpred3 = rbf3.predict(finaltestset)
rpred4 = rbf4.predict(finaltestset)

rprob_y0 = rbf0.predict_proba(finaltestset)
rprob_y1 = rbf1.predict_proba(finaltestset)
rprob_y2 = rbf2.predict_proba(finaltestset)
rprob_y3 = rbf3.predict_proba(finaltestset)
rprob_y4 = rbf4.predict_proba(finaltestset)

# ---------  sigmoid -----------------
sigmoid0 =pd.read_pickle(path+"sigmoid0.pkl")
sigmoid1 = pd.read_pickle(path+"sigmoid1.pkl")
sigmoid2 = pd.read_pickle(path+"sigmoid2.pkl")
sigmoid3 = pd.read_pickle(path+"sigmoid3.pkl")
sigmoid4 = pd.read_pickle(path+"sigmoid4.pkl")

# -1 or 1
spred0 = sigmoid0.predict(finaltestset)
spred1 = sigmoid1.predict(finaltestset)
spred2 = sigmoid2.predict(finaltestset)
spred3 = sigmoid3.predict(finaltestset)
spred4 = sigmoid4.predict(finaltestset)

sprob_y0 = sigmoid0.predict_proba(finaltestset)
sprob_y1 = sigmoid1.predict_proba(finaltestset)
sprob_y2 = sigmoid2.predict_proba(finaltestset)
sprob_y3 = sigmoid3.predict_proba(finaltestset)
sprob_y4 = sigmoid4.predict_proba(finaltestset)

In [ ]:
def prediction(c0, c1, c2, c3, c4, p0, p1, p2, p3, p4):
  predicted = []
  for i in range(len(finaltestset)):
    check = {0:0, 1:0, 2:0, 3:0, 4:0}
    if c0[i] + c1[i] + c2[i] + c3[1] + c4[1] == -5:
      predicted.append(-1)
      print("\n unknown \n ")
    else:
      if c0[i] == 1:
        check[0] = max(p0[i].tolist())
      if c1[i] == 1:
        check[1] = max(p1[i].tolist())
      if c2[i] == 1:
        check[2] = max(p2[i].tolist())
      if c3[i] == 1:
        check[3] = max(p3[i].tolist())
      if c4[i] == 1:
        check[4] = max(p4[i].tolist())

      pos_class = max(check, key=check.get)
      predicted.append(pos_class)
  return predicted

In [ ]:
linearList = prediction(lpred0, lpred1, lpred2, lpred3, lpred4, lprob_y0, lprob_y1, lprob_y2, lprob_y3, lprob_y4)
polyList = prediction(ppred0, ppred1, ppred2, ppred3, ppred4, pprob_y0, pprob_y1, pprob_y2, pprob_y3, pprob_y4)
rbfList = prediction(rpred0, rpred1, rpred2, rpred3, rpred4, rprob_y0, rprob_y1, rprob_y2, rprob_y3, rprob_y4)
sigmoidList = prediction(spred0, spred1, spred2, spred3, spred4, sprob_y0, sprob_y1, sprob_y2, sprob_y3, sprob_y4)

In [ ]:
from sklearn.metrics import f1_score

print("Linear - F1 Scores")
print("---------------")
print("Macro: " + str(f1_score(testY_class, linearList, average="macro", zero_division=1)))
print("Micro: " + str(f1_score(testY_class, linearList, average="micro", zero_division=1)))
print("Weighted :" + str(f1_score(testY_class, linearList, average="weighted", zero_division=1)))
print("None: " + str(f1_score(testY_class, linearList, average=None, zero_division=1)))

print("Poly - F1 Scores")
print("---------------")
print("Macro: " + str(f1_score(testY_class, polyList, average="macro", zero_division=1)))
print("Micro: " + str(f1_score(testY_class, polyList, average="micro", zero_division=1)))
print("Weighted :" + str(f1_score(testY_class, polyList, average="weighted", zero_division=1)))
print("None: " + str(f1_score(testY_class, polyList, average=None, zero_division=1)))

print("RBF - F1 Scores")
print("---------------")
print("Macro: " + str(f1_score(testY_class, rbfList, average="macro", zero_division=1)))
print("Micro: " + str(f1_score(testY_class, rbfList, average="micro", zero_division=1)))
print("Weighted :" + str(f1_score(testY_class, rbfList, average="weighted", zero_division=1)))
print("None: " + str(f1_score(testY_class, rbfList, average=None, zero_division=1)))

print("Sigmoid - F1 Scores")
print("---------------")
print("Macro: " + str(f1_score(testY_class, sigmoidList, average="macro", zero_division=1)))
print("Micro: " + str(f1_score(testY_class, sigmoidList, average="micro", zero_division=1)))
print("Weighted :" + str(f1_score(testY_class, sigmoidList, average="weighted", zero_division=1)))
print("None: " + str(f1_score(testY_class, sigmoidList, average=None, zero_division=1)))

Linear - F1 Scores
---------------
Macro: 0.7438437093099054
Micro: 0.8120868744098206
Weighted :0.7412537242250419
None: [0.         0.83534137 0.94267516 0.91203704 0.90254237 0.87046632]
Poly - F1 Scores
---------------
Macro: 0.7429922031674532
Micro: 0.8139754485363551
Weighted :0.7428656962737212
None: [0.         0.83870968 0.91823899 0.93208431 0.89263158 0.87628866]
RBF - F1 Scores
---------------
Macro: 0.7452626423249907
Micro: 0.8158640226628895
Weighted :0.7448095095512436
None: [0.         0.83534137 0.92405063 0.9342723  0.89873418 0.87917738]
Sigmoid - F1 Scores
---------------
Macro: 0.6769461076423432
Micro: 0.7431539187913125
Weighted :0.6822680333925122
None: [0.         0.71875    0.77124183 0.89260143 0.80769231 0.87139108]


In [ ]:
print(linearList)

In [ ]:
def evaluate(predicted, expected):
    acc = np.mean(np.array(predicted) == np.array(expected))
    print("Overall accuracy: {}".format(acc))
    acc_dict = {}
    for i in range(len(expected)):
        expected_class = expected[i]
        if expected_class not in acc_dict:
            acc_dict[expected_class] = [0, 0]
        acc_dict[expected_class][1] += 1
        if expected_class == predicted[i]:
            acc_dict[expected_class][0] += 1
    for k,v in acc_dict.items():
        print("Accuracy for class {}: {}".format(k, v[0]/v[1]))

evaluate(pred, testY)

# Using features extracted from VGGish

In [ ]:
from tqdm import tqdm

class SpecLoader():
  def __init__(self, x_set, spec_dir):
    self.x = x_set
    self.spec_dir = spec_dir

  def __len__(self):
    return len(self.x)

  def getitem(self):
    specs = []
    for fileName in tqdm(self.x):
        spec = np.load(self.spec_dir + fileName + ".npy")
        specs.append(spec.transpose())
    return np.array(specs)

  def getitemlist(self):
    specs = []
    for fileName in tqdm(self.x):
        spec = np.load(self.spec_dir + fileName + ".npy")
        specs.append(spec.transpose())
    return specs

In [ ]:
from sklearn.model_selection import train_test_split

known_test_size = 0.2
val_test_size = 0.2

vggfeatures_path = os.path.join(root_path,"embeddings/")

data_df = pd.read_csv(csv_path)
data_df_known = data_df.loc[data_df["classID"] < 5]
data_df_unknown = data_df.loc[data_df["classID"] >= 5]


Xtrain_known, Xtest_known, Ytrain_known, Ytest_known = train_test_split(data_df_known['slice_file_name'].tolist(), data_df_known['classID'].tolist(), test_size=known_test_size, random_state = 42)

xtrain0, xtrain1, xtrain2, xtrain3, xtrain4 = [], [], [], [], []

for i in range(len(Xtrain_known)):
  if Ytrain_known[i] == 0:
    xtrain0.append(Xtrain_known[i])
  elif Ytrain_known[i] == 1:
    xtrain1.append(Xtrain_known[i])
  elif Ytrain_known[i] == 2:
    xtrain2.append(Xtrain_known[i])
  elif Ytrain_known[i] == 3:
    xtrain3.append(Xtrain_known[i])
  elif Ytrain_known[i] == 4:
    xtrain4.append(Xtrain_known[i]) 
  else:
    print("cannot separate")

len0 = len(xtrain0)
len1 = len(xtrain1)
len2 = len(xtrain2)
len3 = len(xtrain3)
len4 = len(xtrain4)

xList = xtrain0 + xtrain1 + xtrain2 + xtrain3 + xtrain4

slvgg = SpecLoader(xList, vggfeatures_path)
xtrainsSpec = slvgg.getitemlist()

x0 = xtrainsSpec[:len0]
x1 = xtrainsSpec[len0:len0+len1]
x2 = xtrainsSpec[len0+len1:len0+len1+len2]
x3 = xtrainsSpec[len0+len1+len2:len0+len1+len2+len3]
x4 = xtrainsSpec[len0+len1+len2+len3:]

X0trainval = x0 + x1 + x2 + x3 + x4
Y0trainval = [1 for i in range(len0)]+[ -1 for i in range(len1+len2+len3+len4)]
X1trainval = x1 + x0 + x2 + x3 + x4
Y1trainval = [1 for i in range(len1)]+[-1 for i in range(len0+len2+len3+len4)]
X2trainval = x2 + x0 + x1 + x3 + x4
Y2trainval = [1 for i in range(len2)]+[-1 for i in range(len0+len1+len3+len4)]
X3trainval = x3 + x0 + x2 + x1 + x4
Y3trainval = [1 for i in range(len3)]+[-1 for i in range(len0+len2+len1+len4)]
X4trainval = x4 + x0 + x2 + x3 + x1
Y4trainval = [1 for i in range(len4)]+[-1 for i in range(len0+len2+len3+len1)]


In [ ]:
# converting to numpy
X0train = np.array(X0trainval)
nsamples, x, y = X0train.shape
X0train = X0train.reshape((nsamples, x*y))
Y0train = np.array(Y0trainval)

X1train = np.array(X1trainval)
nsamples, x, y = X1train.shape
X1train = X1train.reshape((nsamples, x*y))
Y1train = np.array(Y1trainval)

X2train = np.array(X2trainval)
nsamples, x, y = X2train.shape
X2train = X2train.reshape((nsamples, x*y))
Y2train = np.array(Y2trainval)

X3train = np.array(X3trainval)
nsamples, x, y = X3train.shape
X3train = X3train.reshape((nsamples, x*y))
Y3train = np.array(Y3trainval)

X4train = np.array(X4trainval)
nsamples, x, y = X4train.shape
X4train = X4train.reshape((nsamples, x*y))
Y4train = np.array(Y4trainval)


In [ ]:
# save pickle file
import pickle
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.externals.joblib import parallel_backend

path = "/content/drive/MyDrive/CDS_project/Team No Name/OneVsAll/vggish/"

kernel = "linear"
param_grid  = {'C': [0.1, 1, 10, 100, 1000], 
              'kernel': [kernel]}  


grid0 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
# with parallel_backend('threading'):
grid0.fit(X0train, Y0train)
pickle.dump(grid0, open(path+kernel+"0.pkl", "wb"))

 
# grid1 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
# grid1.fit(X1train, Y1train)
# pickle.dump(grid1, open(path+kernel+"1.pkl", "wb"))

 
# grid2 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
# grid2.fit(X2train, Y2train)
# pickle.dump(grid2, open(path+kernel+"2.pkl", "wb"))


# grid3 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
# grid3.fit(X3train, Y3train)
# pickle.dump(grid3, open(path+kernel+"3.pkl", "wb"))


# grid4 = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
# grid4.fit(X4train, Y4train)
# pickle.dump(grid4, open(path+kernel+"4.pkl", "wb"))


print("########## Done #############")

## Test and Evalute features from VGGish

In [ ]:
from tqdm import tqdm

class SpecLoader():
  def __init__(self, x_set, spec_dir):
    self.x = x_set
    self.spec_dir = spec_dir

  def __len__(self):
    return len(self.x)

  def getitem(self):
    specs = []
    for fileName in tqdm(self.x):
        spec = np.load(self.spec_dir + fileName + ".npy")
        specs.append(spec.transpose())
    return np.array(specs)

  def getitemlist(self):
    specs = []
    for fileName in tqdm(self.x):
        spec = np.load(self.spec_dir + fileName + ".npy")
        specs.append(spec.transpose())
    return specs

In [ ]:
## testing
### split to get unknwon
X_trash, Xtest_unknown, y_trash, Ytest_unknown = train_test_split(data_df_unknown['slice_file_name'].tolist(), data_df_unknown['classID'].tolist(), test_size=0.2, random_state = 42)

testX = Xtest_unknown + Xtest_known

testY = [-1 for i in Xtest_unknown]+[1 for i in Xtest_known]
testY = np.array(testY)
# testY_class = np.array(testY_class)
testY_class = [-1 for i in Ytest_unknown] + Ytest_known

print(testY.shape)
vggfeatures_path = os.path.join(root_path,"embeddings/")
sltest = SpecLoader(testX, vggfeatures_path)
testX = sltest.getitem()

nsamples, x, y = testX.shape
testX = testX.reshape((nsamples, x*y))

In [ ]:
path = "/content/drive/MyDrive/CDS_project/Team No Name/OneVsAll/vggish/"
# path = "/content/drive/MyDrive/Colab Notebooks/vggtest/"

# # load pickle
lin0 = pd.read_pickle(path+"linear0.pkl")
lin1 = pd.read_pickle(path+"linear1.pkl")
lin2 = pd.read_pickle(path+"linear2.pkl")
lin3 = pd.read_pickle(path+"linear3.pkl")
lin4 = pd.read_pickle(path+"linear4.pkl")

# -1 or 1
lpred0 = lin0.predict(testX)
lpred1 = lin1.predict(testX)
lpred2 = lin2.predict(testX)
lpred3 = lin3.predict(testX)
lpred4 = lin4.predict(testX)

lprob_y0 = lin0.predict_proba(testX)
lprob_y1 = lin1.predict_proba(testX)
lprob_y2 = lin2.predict_proba(testX)
lprob_y3 = lin3.predict_proba(testX)
lprob_y4 = lin4.predict_proba(testX)


# ---------  poly -----------------
poly0 =pd.read_pickle(path+"poly0.pkl")
poly1 = pd.read_pickle(path+"poly1.pkl")
poly2 = pd.read_pickle(path+"poly2.pkl")
poly3 = pd.read_pickle(path+"poly3.pkl")
poly4 = pd.read_pickle(path+"poly4.pkl")

# -1 or 1
ppred0 = poly0.predict(testX)
ppred1 = poly1.predict(testX)
ppred2 = poly2.predict(testX)
ppred3 = poly3.predict(testX)
ppred4 = poly4.predict(testX)

pprob_y0 = poly0.predict_proba(testX)
pprob_y1 = poly1.predict_proba(testX)
pprob_y2 = poly2.predict_proba(testX)
pprob_y3 = poly3.predict_proba(testX)
pprob_y4 = poly4.predict_proba(testX)

# ---------  rbf -----------------
rbf0 =pd.read_pickle(path+"rbf0.pkl")
rbf1 = pd.read_pickle(path+"rbf1.pkl")
rbf2 = pd.read_pickle(path+"rbf2.pkl")
rbf3 = pd.read_pickle(path+"rbf3.pkl")
rbf4 = pd.read_pickle(path+"poly4.pkl")

# -1 or 1
rpred0 = rbf0.predict(testX)
rpred1 = rbf1.predict(testX)
rpred2 = rbf2.predict(testX)
rpred3 = rbf3.predict(testX)
rpred4 = rbf4.predict(testX)

rprob_y0 = rbf0.predict_proba(testX)
rprob_y1 = rbf1.predict_proba(testX)
rprob_y2 = rbf2.predict_proba(testX)
rprob_y3 = rbf3.predict_proba(testX)
rprob_y4 = rbf4.predict_proba(testX)

# ---------  sigmoid -----------------
sigmoid0 =pd.read_pickle(path+"sigmoid0.pkl")
sigmoid1 = pd.read_pickle(path+"sigmoid1.pkl")
sigmoid2 = pd.read_pickle(path+"sigmoid2.pkl")
sigmoid3 = pd.read_pickle(path+"sigmoid3.pkl")
sigmoid4 = pd.read_pickle(path+"sigmoid4.pkl")

# -1 or 1
spred0 = sigmoid0.predict(testX)
spred1 = sigmoid1.predict(testX)
spred2 = sigmoid2.predict(testX)
spred3 = sigmoid3.predict(testX)
spred4 = sigmoid4.predict(testX)

sprob_y0 = sigmoid0.predict_proba(testX)
sprob_y1 = sigmoid1.predict_proba(testX)
sprob_y2 = sigmoid2.predict_proba(testX)
sprob_y3 = sigmoid3.predict_proba(testX)
sprob_y4 = sigmoid4.predict_proba(testX)

In [ ]:
def prediction(c0, c1, c2, c3, c4, p0, p1, p2, p3, p4):
  predicted = []
  for i in range(len(finaltestset)):
    check = {0:0, 1:0, 2:0, 3:0, 4:0}
    if c0[i] + c1[i] + c2[i] + c3[1] + c4[1] == -5:
      predicted.append(-1)
      print("unknown")
    else:
      if c0[i] == 1:
        check[0] = max(p0[i].tolist())
      if c1[i] == 1:
        check[1] = max(p1[i].tolist())
      if c2[i] == 1:
        check[2] = max(p2[i].tolist())
      if c3[i] == 1:
        check[3] = max(p3[i].tolist())
      if c4[i] == 1:
        check[4] = max(p4[i].tolist())

      pos_class = max(check, key=check.get)
      predicted.append(pos_class)
  return predicted

In [ ]:
# linearList = prediction(lpred0, lpred1, lpred2, lpred3, lpred4, lprob_y0, lprob_y1, lprob_y2, lprob_y3, lprob_y4)
polyList = prediction(ppred0, ppred1, ppred2, ppred3, ppred4, pprob_y0, pprob_y1, pprob_y2, pprob_y3, pprob_y4)
rbfList = prediction(rpred0, rpred1, rpred2, rpred3, rpred4, rprob_y0, rprob_y1, rprob_y2, rprob_y3, rprob_y4)
sigmoidList = prediction(spred0, spred1, spred2, spred3, spred4, sprob_y0, sprob_y1, sprob_y2, sprob_y3, sprob_y4)

In [ ]:
def evaluate(predicted, expected):
    acc = np.mean(np.array(predicted) == np.array(expected))
    print("Overall accuracy: {}".format(acc))
    acc_dict = {}
    for i in range(len(expected)):
        expected_class = expected[i]
        if expected_class not in acc_dict:
            acc_dict[expected_class] = [0, 0]
        acc_dict[expected_class][1] += 1
        if expected_class == predicted[i]:
            acc_dict[expected_class][0] += 1
    for k,v in acc_dict.items():
        print("Accuracy for class {}: {}".format(k, v[0]/v[1]))
    for average in ["macro", "weighted", "micro"]:
        f1 = f1_score(expected, predicted, average=average)
        print("{} f1 score: {}".format(average, f1))


# evaluate(linearList, testY_class)
print("-----------------")
print("Poly:")
print("-----------------")
evaluate(polyList, testY_class)
print("-----------------")
print("RBF:")
print("-----------------")
evaluate(rbfList, testY_class)
print("-----------------")
print("Sigmoid:")
print("-----------------")
evaluate(sigmoidList, testY_class)